<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Intialize-dataset" data-toc-modified-id="Intialize-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Intialize dataset</a></span></li><li><span><a href="#Compute-min/max" data-toc-modified-id="Compute-min/max-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compute min/max</a></span></li><li><span><a href="#Create-Cluster" data-toc-modified-id="Create-Cluster-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Cluster</a></span></li><li><span><a href="#Compute-Histogram-for-CLT-variable" data-toc-modified-id="Compute-Histogram-for-CLT-variable-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compute Histogram for CLT variable</a></span></li><li><span><a href="#Display-results-with-VCS" data-toc-modified-id="Display-results-with-VCS-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Display results with VCS</a></span></li></ul></div>

# Import packages

Use conda to install distributed and dask in your environmnent.
  * **conda install -c conda-forge distributed dask**

In [ ]:
import cdms2
import cdat_info
import dask.array as da
from dask.distributed import LocalCluster
from dask.distributed  import Client
import vcs
import vcsaddons

# Intialize dataset
Convert File Varible into a Dask Array

In [ ]:
f = cdms2.open(cdat_info.get_sampledata_path()+"/clt.nc")
dataFV=f["clt"]
myDaskArray= da.from_array(dataFV, chunks=(1,46,72))

In [ ]:
myDaskArray

# Compute min/max

In [ ]:
min = myDaskArray.min()
max = myDaskArray.max()
print(min.compute(), max.compute())

# Create Cluster

In [ ]:
cluster=LocalCluster(n_workers=5,threads_per_worker=2)
display(cluster)
c=Client(cluster)
c

# Compute Histogram for CLT variable

In [ ]:
f = cdms2.open(cdat_info.get_sampledata_path()+"/clt.nc")
dataFV=f["clt"]
myDaskArray= da.from_array(dataFV, chunks=(1,46,72))
h,bins=da.histogram(myDaskArray,bins=100,range=[1,100])
bins
results = h.compute()

# Display results with VCS

In [ ]:
plt=vcs.init()
plt.plot(bins,results)